# Prompt Engineering

## Setup
Add a .env file with following content:
```env
OPENAI_API_KEY=<Your key>
```

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Read BPMN Diagrams

#### Anhang 1 - Application Process
<div>
    <img src="./images/application.png" width="900"/>
</div>

In [3]:
reader = open("bpmn/application_process.bpmn")
application = reader.read()

#### Anhang 2 - Cooking Process
<div>
    <img src="./images/cooking.png" width="900"/>
</div>

In [4]:
reader = open("bpmn/cooking.bpmn")
cooking = reader.read()

#### Anhang 3 - Invoice Process
<div>
    <img src="./images/invoice.png" width="900"/>
</div>

In [5]:
reader = open("bpmn/invoice.bpmn")
invoice = reader.read()

## Anhang 4 - How well does ChatGPT understand BPMN?

### Question to the BPMN 2.0 Specification

In [14]:
prompt = f"""
List all different task types of the BPMN 2.0 specification.
"""
response = get_completion(prompt)
print(response)

1. Task
2. User Task
3. Manual Task
4. Service Task
5. Business Rule Task
6. Script Task
7. Send Task
8. Receive Task
9. Call Activity
10. Sub-Process Task
11. Event Sub-Process Task
12. Transaction Task
13. Ad-Hoc Sub-Process Task
14. Compensation Task
15. Gateway Task
16. Exclusive Gateway
17. Inclusive Gateway
18. Parallel Gateway
19. Complex Gateway
20. Event-Based Gateway.


#### Result:
* Wrong answer Gateways or Sub-Processes are not a Task Type

### Describe a given BPMN Diagram

In [12]:
prompt = f"""
Your task is to describe a given BPMN Diagram.
```{application}```
"""
response = get_completion(prompt)
print(response)

The BPMN diagram represents a process for filling a job vacancy. The process starts with a "Vacancy emerged" event and then moves to a "Report vacancy" user task. Once the vacancy is reported, the process moves to a "Create job posting" user task. After the job posting is created, the process moves to a "Application received" service task. Once the application is received, the process moves to a "Hold application exam" user task. After the exam is held, the process moves to a "Select applicant" user task. Finally, the process ends with a "Position filled" end event. The diagram includes sequence flows connecting the events and tasks, and BPMN shapes representing the events and tasks.


In [55]:
prompt = f"""
Your task is to describe a given BPMN Diagram.
```{cooking}```
"""
response = get_completion(prompt)
print(response)

The given BPMN diagram represents a process for choosing and preparing a meal. Here is a description of the diagram:

The process starts with a "Hunger noted" event, represented by a start event. From this event, a sequence flow leads to the "Choose recipe" user task, represented by a rectangle with the name "Choose recipe". After the user task, another sequence flow leads to an exclusive gateway with the name "Desired dish?". This gateway has two outgoing sequence flows, one named "Pasta" and the other named "Steak".

If the "Pasta" sequence flow is taken, it leads to the "Cook pasta" user task. If the "Steak" sequence flow is taken, it leads to the "Fry steak" user task. Both user tasks are represented by rectangles with their respective names.

After the "Cook pasta" and "Fry steak" user tasks, the process converges at another exclusive gateway, represented by a diamond shape. This gateway has two incoming sequence flows, one from the "Cook pasta" user task and the other from the "F

In [56]:
prompt = f"""
Your task is to describe a given BPMN Diagram.
```{invoice}```
"""
response = get_completion(prompt)
print(response)

The given BPMN diagram represents a process flow for handling the sale of an item and the associated invoicing process. Here is a description of the diagram:

1. The process starts with an event called "Item sold" (Start Event). This event signifies the initiation of the process when an item is sold.

2. From the start event, there is a sequence flow that connects to a user task called "Create invoice" (User Task). This task represents the creation of an invoice for the sold item.

3. After the "Create invoice" task, there is another sequence flow that connects to a service task called "Send invoice" (Service Task). This task represents the action of sending the created invoice to the customer.

4. From the "Send invoice" task, there is a sequence flow that connects to an exclusive gateway called "Received money?" (Exclusive Gateway). This gateway represents a decision point based on whether the money for the invoice has been received or not.

5. The "Received money?" gateway has two i

#### Result:
* Extracted the correct:
    * Process meaning (e.g. "filling a job vacancy")
    * Element Name (e.g. "Vacancy emerged")
    * Task Type (e.g User Task)
    * Sequence Flow
* Include unnecessary information (e.g. the shape of a task or gateway)
* Different output formats over the three prompts

## Anhang 5 - Describing the BPMN process

In [10]:
prompt_1 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
Whenever I mention "relevant" BPMN elements I mean all BPMN elements described in the official specification excluding the sequence flow.
Describe the process as a list of items.
Every item represents a relevant BPMN element.
The list has the following structure:

1. <ELEMENT> "<NAME>" -> 2.
2. <ELEMENT> "<NAME>" -> 3.
...
N. <ELEMENT> "<NAME>"

Replace <ELEMENT> with the relevant BPMN element (e.g. Startevent, Exclusive Gateway, etc.).
Replace <NAME> with the name attribute in the given XML for the corresponding element.
The first number represents the position of that element.
The number after the arrow (->) represents the next element.

A Gateway can have multiple following elements. If that is the case separate the numbers with a comma.
It is important that the final list do not include any sequence flow.

```{application}```
"""
response_1 = get_completion(prompt_1)
print(response_1)

1. StartEvent "Vacancy emerged" -> 2
2. UserTask "Report vacancy" -> 3
3. UserTask "Create job posting" -> 4
4. ServiceTask "Application received" -> 5
5. UserTask "Hold application exam" -> 6
6. UserTask "Select applicant" -> 7
7. EndEvent "Position filled"


In [11]:
prompt_2 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
From now on, whenever I mention "relevant" BPMN elements I mean all BPMN elements described in the official specification excluding the sequence flow.
Describe the process as a list of items.
Every item represents a relevant BPMN element.
The list has the following structure:

1. <ELEMENT> "<NAME>" -> 2.
2. <ELEMENT> "<NAME>" -> 3.
...
N. <ELEMENT> "<NAME>"

Replace <ELEMENT> with the relevant BPMN element (e.g. Startevent, Exclusive Gateway, etc.).
Replace <NAME> with the name attribute in the given XML for the corresponding element. If the attribute name is missing then ignore <NAME>.
The first number represents the position of that element and acts like an id for the element.
The number after the arrow (->) represents the id of the next element.

A Gateway can have multiple following elements. If that is the case separate the numbers with a comma.
It is important that the final list do not include any sequence flow.

```{cooking}```
"""
response = get_completion(prompt_2)
print(response)

1. StartEvent "Hunger noted" -> 2
2. SequenceFlow -> 3
3. ExclusiveGateway "Desired dish?" -> 4,5
4. SequenceFlow -> 6
5. SequenceFlow -> 7
6. UserTask "Choose recipe" -> 8
7. UserTask "Fry steak" -> 8
8. ExclusiveGateway -> 9
9. SequenceFlow -> 10
10. SequenceFlow -> 11
11. UserTask "Cook pasta" -> 12
12. UserTask "Eat meal" -> 13
13. EndEvent "Hunger satisfied"


In [14]:
prompt_3 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
From now on, whenever I mention "relevant" BPMN elements I mean all BPMN elements described in the official specification excluding the sequence flow.
Describe the process as a list of items.
Every item represents a relevant BPMN element.
The list has the following structure:

1. <ELEMENT> "<NAME>" -> 2.
2. <ELEMENT> "<NAME>" -> 3.
...
N. <ELEMENT> "<NAME>"

Replace <ELEMENT> with the relevant BPMN element (e.g. Startevent, Exclusive Gateway, etc.).
Replace <NAME> with the name attribute in the given XML for the corresponding element. If the attribute name is missing then ignore <NAME>.
The first number represents the position of that element and acts like an id for the element.
The number after the arrow (->) represents the id of the next element.

A Gateway can have multiple following elements. If that is the case separate the numbers with a comma.
It is important that the final list do not include any sequence flow.

```{invoice}```
"""
response = get_completion(prompt_3)
print(response)

1. StartEvent "Item sold" -> 2
2. SequenceFlow -> 3
3. UserTask "Create invoice" -> 4
4. SequenceFlow -> 5
5. ServiceTask "Send invoice" -> 6
6. SequenceFlow -> 7
7. ExclusiveGateway "Received money?" -> 8,9
8. SequenceFlow -> 10
9. SequenceFlow -> 8,9
10. ServiceTask "Send reminder" -> 11
11. SequenceFlow -> 12
12. ExclusiveGateway "Received money?" -> 13,14
13. SequenceFlow -> 15
14. SequenceFlow -> 13,14
15. EndEvent "Invoice settled"


In [13]:
prompt_4 = f"""
Your task is to answer in a consistent style.

<person>: {prompt_1}

<chatgpt>: {response_1}

<person>: {prompt_2}
"""
response = get_completion(prompt_4, "gpt-3.5-turbo-16k")
print(response)


<chatgpt>: 1. StartEvent "Hunger noted" -> 2
2. UserTask "Choose recipe" -> 3
3. ExclusiveGateway "Desired dish?" -> 4,5
4. UserTask "Cook pasta" -> 6
5. UserTask "Fry steak" -> 6
6. ExclusiveGateway -> 7
7. UserTask "Eat meal" -> 8
8. EndEvent "Hunger satisfied"


In [15]:
prompt_5 = f"""
Your task is to answer in a consistent style.

<person>: {prompt_1}

<chatgpt>: {response_1}

<person>: {prompt_3}
"""
response = get_completion(prompt_5, "gpt-3.5-turbo-16k")
print(response)


<chatgpt>: 1. StartEvent "Item sold" -> 2
2. UserTask "Create invoice" -> 3
3. ServiceTask "Send invoice" -> 4
4. ExclusiveGateway "Received money?" -> 5,6
5. ServiceTask "Send reminder" -> 7
6. EndEvent "Invoice settled"
7. EndEvent "Invoice settled"


#### Result:
* *prompt_1* leads to the correct output without sequence flows
* *prompt_2* somehow does not remove the sequence flows sufficiently
    * even after multiple tries with different prompts
    * does not always create a correct list
* *prompt_3* incorrect process description (11. is never reached)
* *prompt_4* uses few shot prompting
    * solves the problem with the appearance or sequence flows in the final list
    * needs a different model with more tokens
    * output for third BPMN diagram is wrong


## Anhang 6 - Extracting information


In [9]:
processes = [application, cooking, invoice]
for i in range(len(processes)):
    prompt = f"""
    Your task is to generate a short summary of a given BPMN Process. \

    Summarize the process below, delimited by triple
    backticks, in at most 1 sentence.

    Process: ```{processes[i]}```
    """
    response = get_completion(prompt)
    print(i, response, "\n")


0 The given BPMN process involves the steps of reporting a vacancy, creating a job posting, receiving applications, holding an application exam, selecting an applicant, and filling the position. 

1 The process involves choosing a recipe based on hunger, cooking the chosen dish, and then eating the meal to satisfy hunger. 

2 The given BPMN process involves the steps of creating an invoice, sending the invoice, checking if money is received, and sending a reminder if necessary, with the process ending when the invoice is settled. 



In [11]:
prompt = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
Extract all distinct BPMN elements present in the diagram, excluding sequence flows.
Also count how often a element is used.
Display the list in the following format:

* <ELEMENT> <COUNT>
* <ELEMENT> <COUNT>
...

Replace <ELEMENT> with the relevant BPMN element (e.g. StartEvent, Exclusive Gateway, etc.)
Replace <COUNT> with the number of times the element is contained in the diagram.

Note that the sequence flows will be excluded from the list, as per my instructions.

```{application}```
"""
response = get_completion(prompt)
print(response)

* bpmn:definitions 1
* bpmn:process 1
* bpmn:startEvent 1
* bpmn:sequenceFlow 5
* bpmn:userTask 4
* bpmn:serviceTask 1
* bpmn:endEvent 1


### Search for loop or cycle

In [16]:
prompt_1 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
Determine if the diagram contains a loop or cycle in which one or more tasks can be executed multiple times.
Provide an answer limited to two words "Yes" or "No".

```{cooking}```
"""
response = get_completion(prompt_1)
print(response)

No


In [17]:
prompt_2 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
Determine if the diagram contains a loop or cycle in which one or more tasks can be executed multiple times.
Provide an answer limited to two words "Yes" or "No".

```{invoice}```
"""
response = get_completion(prompt_2)
print(response)

No


In [18]:
prompt_3 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks.
Determine if the diagram contains a loop or cycle in which one or more tasks can be executed multiple times.

```{invoice}```
"""
response = get_completion(prompt_3)
print(response)


Yes, the BPMN diagram contains a loop or cycle. The loop is formed by the sequence flow between the "Send reminder" task (Activity_0mgn8rb) and the exclusive gateway (Gateway_16w5r3m), and the sequence flow between the exclusive gateway and the "Send invoice" task (Activity_0zlx5ya). This loop allows the "Send reminder" and "Send invoice" tasks to be executed multiple times.


In [20]:
prompt_4 = f"""
You will be provided with an BPMN Diagram as XML delimited by triple backticks. \
Determine if the diagram contains a loop or cycle in which one or more BPMN elements are executed multiple times. \
Don't decide until you followed every possible sequence flow and gateway. \
Perform the following actions: \
1 - Simulate a token. \
  - The token have to follow every possible path step-by-step. \
  - If a Gateway is visited twice simulate each outgoing sequence flow at least twice. \
2 - Create a list with the visited BPMN element of every simulation step. \
3 - Check if the resulting list contains duplicates. \
4 - Provide an answer.

Use the following format: \
Visited elements: \
<list of visited elements> \
Duplicates: \
<duplicates> \
Answer: \
<Yes or No>

```{invoice}```
"""
response = get_completion(prompt_4)
print(response)

Visited elements: [Event_140mb0o, Flow_1px778j, Activity_0rllx8f, Flow_1yc3h6k, Activity_0zlx5ya, Flow_1dcej3e, Gateway_16w5r3m, Flow_09cpp6w, Activity_0mgn8rb, Flow_10vlhhj, Gateway_16w5r3m, Flow_1gcqri7, Event_0rp17pv]
Duplicates: Gateway_16w5r3m
Answer: Yes


#### Result
* The output format influence if the answer is correct or incorrect

## Anhang 7 - Flipped Interaction

In [2]:
from IPython.display import Markdown, display
display(Markdown("./images/flipped_interaction.md"))

**You:**

From now on, I would like you to ask me questions to create a executable BPMN 2.0 Diagram. When you have enough information to create the BPMN Diagram give me the XML for this diagram.
Ask me the questions one at a time.

------------------

**ChatGPT:**

Sure, I can help you create an executable BPMN 2.0 diagram. Let's start with the first question:

1. What is the process you want to model using BPMN? Please provide a brief description or name for the process.



------------------

**You:**

1. The process is about the sale of an item and the associated invoicing process.

------------------

**ChatGPT:**

Great! Now let's move on to the next question:

1. What are the main steps or activities involved in the sales and invoicing process?



------------------

**You:**

Steps are as follows:
1. Item sold
2. Create invoice
3. Send invoice
4. Wait for payment
If we get the money
5. Process ends
Else
5. Wait for 1 week
6. Send a reminder

------------------

**ChatGPT:**

Thank you for providing the steps. Now, let's continue with the next question:

1. Are there any specific roles or participants involved in the process? If so, please provide the names or descriptions of the roles.



------------------

**You:**

No please create the process without specific roles.

------------------

**ChatGPT:**

Understood. Moving on to the next question:

1. Are there any specific events that trigger or occur during the sales and invoicing process? Examples of events could be "Item Sold" or "Payment Received." If there are any events, please provide their names or descriptions.



------------------

**You:**

"Item Sold" is a Start Event.
"Wait for payment" is a Timer Event.
"Wait 1 week" is another Timer Event.
"Payment received" is the End Event.

------------------

**ChatGPT:**

Got it. Here's the summary of the information you provided:
* Start Event: "Item Sold"* Activities:

</li>* End Event: "Payment Received"Now, let's proceed to the next question:

1. Are there any conditions or gateways that determine the flow of the process? For example, any decision points where the process takes different paths based on certain conditions. If there are any conditions or gateways, please describe them.



------------------

**You:**

Yes, we need one 

------------------

**ChatGPT:**

Understood. Please provide a description or name for the condition or gateway, and any specific conditions or rules that determine the flow of the process at that point.


------------------

**You:**

The Gateway has to be after "Wait for Payment". The Gateway has to outgoing sequence flows. One sequence flow is labeled with "Yes" and means that we received the money. The other sequence flow is labeled with "No" and means we have to "Send Reminder".

------------------

**ChatGPT:**

Thank you for providing the details. Here's the updated summary of the process:
* Start Event: "Item Sold"* Activities:

</li>  * Gateway: After "Wait for Payment" with two outgoing sequence flows
  * Sequence Flow 1: Labeled as "Yes" (Payment received)  * Sequence Flow 2: Labeled as "No" (Send Reminder)</li><li>Activities:
4. Wait 1 Week (Connected to "No" sequence flow)
  1. Send Reminder (Connected to "No" sequence flow)

</li>
- End Event: "Payment Received"Now, let's proceed to the final question:

  1. Are there any additional details or requirements you would like to include in the BPMN diagram? This could be specific events, timers, message flows, or any other relevant information. If there are no additional requirements, please let me know.



------------------

**You:**

There are no additional requirements.

------------------

**ChatGPT:**

Great! Based on the information you provided, I have created the BPMN diagram for the sales and invoicing process. Here's the corresponding BPMN XML code:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<bpmn2:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" id="Definitions_1" targetNamespace="http://bpmn.io/schema/bpmn">
  <bpmn2:process id="Process_1" isExecutable="true">
    <bpmn2:startEvent id="StartEvent_1">
      <bpmn2:outgoing>SequenceFlow_1</bpmn2:outgoing>
    </bpmn2:startEvent>
    <bpmn2:task id="Task_1" name="Create Invoice">
      <bpmn2:incoming>SequenceFlow_1</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_2</bpmn2:outgoing>
    </bpmn2:task>
    <bpmn2:task id="Task_2" name="Send Invoice">
      <bpmn2:incoming>SequenceFlow_2</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_3</bpmn2:outgoing>
    </bpmn2:task>
    <bpmn2:intermediateCatchEvent id="IntermediateCatchEvent_1">
      <bpmn2:timerEventDefinition/>
      <bpmn2:incoming>SequenceFlow_3</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_4</bpmn2:outgoing>
    </bpmn2:intermediateCatchEvent>
    <bpmn2:exclusiveGateway id="ExclusiveGateway_1">
      <bpmn2:incoming>SequenceFlow_4</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_5</bpmn2:outgoing>
      <bpmn2:outgoing>SequenceFlow_6</bpmn2:outgoing>
    </bpmn2:exclusiveGateway>
    <bpmn2:intermediateCatchEvent id="IntermediateCatchEvent_2">
      <bpmn2:timerEventDefinition/>
      <bpmn2:incoming>SequenceFlow_6</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_7</bpmn2:outgoing>
    </bpmn2:intermediateCatchEvent>
    <bpmn2:task id="Task_3" name="Send Reminder">
      <bpmn2:incoming>SequenceFlow_7</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_8</bpmn2:outgoing>
    </bpmn2:task>
    <bpmn2:endEvent id="EndEvent_1">
      <bpmn2:incoming>SequenceFlow_5</bpmn2:incoming>
    </bpmn2:endEvent>
    <bpmn2:endEvent id="EndEvent_2">
      <bpmn2:incoming>SequenceFlow_8</bpmn2:incoming>
    </bpmn2:endEvent>
    <bpmn2:sequenceFlow id="SequenceFlow_1" sourceRef="StartEvent_1" targetRef="Task_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_2" sourceRef="Task_1" targetRef="Task_2"/>
    <bpmn2:sequenceFlow id="SequenceFlow_3" sourceRef="Task_2" targetRef="IntermediateCatchEvent_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_4" sourceRef="IntermediateCatchEvent_1" targetRef="ExclusiveGateway_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_5" sourceRef="ExclusiveGateway_1" targetRef="EndEvent_1">
      <bpmn2:conditionExpression xsi:type="bpmn2:tFormalExpression">#{approved}</bpmn2:conditionExpression>
    </bpmn2:sequenceFlow>
    <bpmn2:sequenceFlow id="SequenceFlow_6" sourceRef="ExclusiveGateway_1" targetRef="IntermediateCatchEvent_2">
      <bpmn2:conditionExpression xsi:type="bpmn2:tFormalExpression">#{!approved}</bpmn2:conditionExpression>
    </bpmn2:sequenceFlow>
    <bpmn2:sequenceFlow id="SequenceFlow_7" sourceRef="IntermediateCatchEvent_2" targetRef="Task_3"/>
    <bpmn2:sequenceFlow id="SequenceFlow_8" sourceRef="Task_3" targetRef="EndEvent_2"/>
  </bpmn2:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_1">
      <bpmndi:BPMNShape id="StartEvent_1_di" bpmnElement="StartEvent_1">
        <dc:Bounds x="100" y="100" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_1_di" bpmnElement="Task_1">
        <dc:Bounds x="200" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_2_di" bpmnElement="Task_2">
        <dc:Bounds x="350" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="IntermediateCatchEvent_1_di" bpmnElement="IntermediateCatchEvent_1">
        <dc:Bounds x="510" y="80" width="36" height="36"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="523" y="123" width="40" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="ExclusiveGateway_1_di" bpmnElement="ExclusiveGateway_1" isMarkerVisible="true">
        <dc:Bounds x="600" y="80" width="50" height="50"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="613" y="123" width="24" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="IntermediateCatchEvent_2_di" bpmnElement="IntermediateCatchEvent_2">
        <dc:Bounds x="680" y="80" width="36" height="36"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="682" y="123" width="32" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_3_di" bpmnElement="Task_3">
        <dc:Bounds x="760" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="EndEvent_1_di" bpmnElement="EndEvent_1">
        <dc:Bounds x="890" y="98" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="EndEvent_2_di" bpmnElement="EndEvent_2">
        <dc:Bounds x="890" y="163" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="SequenceFlow_1_di" bpmnElement="SequenceFlow_1">
        <di:waypoint xsi:type="dc:Point" x="136" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="200" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_2_di" bpmnElement="SequenceFlow_2">
        <di:waypoint xsi:type="dc:Point" x="280" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="350" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_3_di" bpmnElement="SequenceFlow_3">
        <di:waypoint xsi:type="dc:Point" x="430" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="510" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_4_di" bpmnElement="SequenceFlow_4">
        <di:waypoint xsi:type="dc:Point" x="546" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="105"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="130"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_5_di" bpmnElement="SequenceFlow_5">
        <di:waypoint xsi:type="dc:Point" x="650" y="105"/>
        <di:waypoint xsi:type="dc:Point" x="890" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_6_di" bpmnElement="SequenceFlow_6">
        <di:waypoint xsi:type="dc:Point" x="650" y="130"/>
        <di:waypoint xsi:type="dc:Point" x="680" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_7_di" bpmnElement="SequenceFlow_7">
        <di:waypoint xsi:type="dc:Point" x="716" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="760" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_8_di" bpmnElement="SequenceFlow_8">
        <di:waypoint xsi:type="dc:Point" x="840" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="890" y="118"/>
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn2:definitions>

```
Please note that the XML provided above represents the structure and elements of the BPMN diagram based on the information you provided. You can use this XML code to import and execute the BPMN diagram in BPMN-compliant tools or engines.


------------------

**You:**

Thank you this process already looks very good. But after "Send Reminder" the outgoing sequence flow should go back to the Gateway. This way we have a loop and can execute it multiple times.

------------------

**ChatGPT:**

Apologies for the oversight. I have updated the BPMN diagram to include the loopback connection from the "Send Reminder" task back to the gateway. Here's the revised BPMN XML code:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<bpmn2:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" id="Definitions_1" targetNamespace="http://bpmn.io/schema/bpmn">
  <bpmn2:process id="Process_1" isExecutable="true">
    <bpmn2:startEvent id="StartEvent_1">
      <bpmn2:outgoing>SequenceFlow_1</bpmn2:outgoing>
    </bpmn2:startEvent>
    <bpmn2:task id="Task_1" name="Create Invoice">
      <bpmn2:incoming>SequenceFlow_1</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_2</bpmn2:outgoing>
    </bpmn2:task>
    <bpmn2:task id="Task_2" name="Send Invoice">
      <bpmn2:incoming>SequenceFlow_2</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_3</bpmn2:outgoing>
    </bpmn2:task>
    <bpmn2:intermediateCatchEvent id="IntermediateCatchEvent_1">
      <bpmn2:timerEventDefinition/>
      <bpmn2:incoming>SequenceFlow_3</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_4</bpmn2:outgoing>
    </bpmn2:intermediateCatchEvent>
    <bpmn2:exclusiveGateway id="ExclusiveGateway_1">
      <bpmn2:incoming>SequenceFlow_4</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_5</bpmn2:outgoing>
      <bpmn2:outgoing>SequenceFlow_6</bpmn2:outgoing>
    </bpmn2:exclusiveGateway>
    <bpmn2:intermediateCatchEvent id="IntermediateCatchEvent_2">
      <bpmn2:timerEventDefinition/>
      <bpmn2:incoming>SequenceFlow_6</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_7</bpmn2:outgoing>
    </bpmn2:intermediateCatchEvent>
    <bpmn2:task id="Task_3" name="Send Reminder">
      <bpmn2:incoming>SequenceFlow_7</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_8</bpmn2:outgoing>
    </bpmn2:task>
    <bpmn2:endEvent id="EndEvent_1">
      <bpmn2:incoming>SequenceFlow_5</bpmn2:incoming>
    </bpmn2:endEvent>
    <bpmn2:sequenceFlow id="SequenceFlow_1" sourceRef="StartEvent_1" targetRef="Task_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_2" sourceRef="Task_1" targetRef="Task_2"/>
    <bpmn2:sequenceFlow id="SequenceFlow_3" sourceRef="Task_2" targetRef="IntermediateCatchEvent_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_4" sourceRef="IntermediateCatchEvent_1" targetRef="ExclusiveGateway_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_5" sourceRef="ExclusiveGateway_1" targetRef="EndEvent_1">
      <bpmn2:conditionExpression xsi:type="bpmn2:tFormalExpression">#{approved}</bpmn2:conditionExpression>
    </bpmn2:sequenceFlow>
    <bpmn2:sequenceFlow id="SequenceFlow_6" sourceRef="ExclusiveGateway_1" targetRef="IntermediateCatchEvent_2">
      <bpmn2:conditionExpression xsi:type="bpmn2:tFormalExpression">#{!approved}</bpmn2:conditionExpression>
    </bpmn2:sequenceFlow>
    <bpmn2:sequenceFlow id="SequenceFlow_7" sourceRef="IntermediateCatchEvent_2" targetRef="Task_3"/>
    <bpmn2:sequenceFlow id="SequenceFlow_8" sourceRef="Task_3" targetRef="ExclusiveGateway_1"/>
  </bpmn2:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_1">
      <bpmndi:BPMNShape id="StartEvent_1_di" bpmnElement="StartEvent_1">
        <dc:Bounds x="100" y="100" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_1_di" bpmnElement="Task_1">
        <dc:Bounds x="200" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_2_di" bpmnElement="Task_2">
        <dc:Bounds x="350" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="IntermediateCatchEvent_1_di" bpmnElement="IntermediateCatchEvent_1">
        <dc:Bounds x="510" y="80" width="36" height="36"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="523" y="123" width="40" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="ExclusiveGateway_1_di" bpmnElement="ExclusiveGateway_1" isMarkerVisible="true">
        <dc:Bounds x="600" y="80" width="50" height="50"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="613" y="123" width="24" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="IntermediateCatchEvent_2_di" bpmnElement="IntermediateCatchEvent_2">
        <dc:Bounds x="680" y="80" width="36" height="36"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="682" y="123" width="32" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_3_di" bpmnElement="Task_3">
        <dc:Bounds x="760" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="EndEvent_1_di" bpmnElement="EndEvent_1">
        <dc:Bounds x="890" y="98" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="EndEvent_2_di" bpmnElement="EndEvent_2">
        <dc:Bounds x="890" y="163" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="SequenceFlow_1_di" bpmnElement="SequenceFlow_1">
        <di:waypoint xsi:type="dc:Point" x="136" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="200" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_2_di" bpmnElement="SequenceFlow_2">
        <di:waypoint xsi:type="dc:Point" x="280" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="350" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_3_di" bpmnElement="SequenceFlow_3">
        <di:waypoint xsi:type="dc:Point" x="430" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="510" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_4_di" bpmnElement="SequenceFlow_4">
        <di:waypoint xsi:type="dc:Point" x="546" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="105"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="130"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_5_di" bpmnElement="SequenceFlow_5">
        <di:waypoint xsi:type="dc:Point" x="650" y="105"/>
        <di:waypoint xsi:type="dc:Point" x="890" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_6_di" bpmnElement="SequenceFlow_6">
        <di:waypoint xsi:type="dc:Point" x="650" y="130"/>
        <di:waypoint xsi:type="dc:Point" x="680" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_7_di" bpmnElement="SequenceFlow_7">
        <di:waypoint xsi:type="dc:Point" x="716" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="760" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_8_di" bpmnElement="SequenceFlow_8">
        <di:waypoint xsi:type="dc:Point" x="840" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="890" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_9_di" bpmnElement="SequenceFlow_9">
        <di:waypoint xsi:type="dc:Point" x="790" y="160"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="160"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="105"/>
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn2:definitions>

```
The updated BPMN diagram includes the loopback connection from the "Send Reminder" task back to the exclusive gateway. This allows the process to be executed multiple times in case the payment is still not received.


------------------

**You:**

This is great! Now specifiy the types of the tasks. Instead of the Abstract Task use specific types (e.g. User Task, Service Task, etc.). Decide for yourself which type fits best.

------------------

**ChatGPT:**

Certainly! I will update the task types based on their functionality. Here's the revised BPMN XML code with task types specified:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<bpmn2:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" id="Definitions_1" targetNamespace="http://bpmn.io/schema/bpmn">
  <bpmn2:process id="Process_1" isExecutable="true">
    <bpmn2:startEvent id="StartEvent_1">
      <bpmn2:outgoing>SequenceFlow_1</bpmn2:outgoing>
    </bpmn2:startEvent>
    <bpmn2:userTask id="Task_1" name="Create Invoice">
      <bpmn2:incoming>SequenceFlow_1</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_2</bpmn2:outgoing>
    </bpmn2:userTask>
    <bpmn2:userTask id="Task_2" name="Send Invoice">
      <bpmn2:incoming>SequenceFlow_2</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_3</bpmn2:outgoing>
    </bpmn2:userTask>
    <bpmn2:intermediateCatchEvent id="IntermediateCatchEvent_1">
      <bpmn2:timerEventDefinition/>
      <bpmn2:incoming>SequenceFlow_3</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_4</bpmn2:outgoing>
    </bpmn2:intermediateCatchEvent>
    <bpmn2:exclusiveGateway id="ExclusiveGateway_1">
      <bpmn2:incoming>SequenceFlow_4</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_5</bpmn2:outgoing>
      <bpmn2:outgoing>SequenceFlow_6</bpmn2:outgoing>
    </bpmn2:exclusiveGateway>
    <bpmn2:intermediateCatchEvent id="IntermediateCatchEvent_2">
      <bpmn2:timerEventDefinition/>
      <bpmn2:incoming>SequenceFlow_6</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_7</bpmn2:outgoing>
    </bpmn2:intermediateCatchEvent>
    <bpmn2:userTask id="Task_3" name="Send Reminder">
      <bpmn2:incoming>SequenceFlow_7</bpmn2:incoming>
      <bpmn2:outgoing>SequenceFlow_8</bpmn2:outgoing>
    </bpmn2:userTask>
    <bpmn2:endEvent id="EndEvent_1">
      <bpmn2:incoming>SequenceFlow_5</bpmn2:incoming>
    </bpmn2:endEvent>
    <bpmn2:sequenceFlow id="SequenceFlow_1" sourceRef="StartEvent_1" targetRef="Task_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_2" sourceRef="Task_1" targetRef="Task_2"/>
    <bpmn2:sequenceFlow id="SequenceFlow_3" sourceRef="Task_2" targetRef="IntermediateCatchEvent_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_4" sourceRef="IntermediateCatchEvent_1" targetRef="ExclusiveGateway_1"/>
    <bpmn2:sequenceFlow id="SequenceFlow_5" sourceRef="ExclusiveGateway_1" targetRef="EndEvent_1">
      <bpmn2:conditionExpression xsi:type="bpmn2:tFormalExpression">#{approved}</bpmn2:conditionExpression>
    </bpmn2:sequenceFlow>
    <bpmn2:sequenceFlow id="SequenceFlow_6" sourceRef="ExclusiveGateway_1" targetRef="IntermediateCatchEvent_2">
      <bpmn2:conditionExpression xsi:type="bpmn2:tFormalExpression">#{!approved}</bpmn2:conditionExpression>
    </bpmn2:sequenceFlow>
    <bpmn2:sequenceFlow id="SequenceFlow_7" sourceRef="IntermediateCatchEvent_2" targetRef="Task_3"/>
    <bpmn2:sequenceFlow id="SequenceFlow_8" sourceRef="Task_3" targetRef="ExclusiveGateway_1"/>
  </bpmn2:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_1">
      <bpmndi:BPMNShape id="StartEvent_1_di" bpmnElement="StartEvent_1">
        <dc:Bounds x="100" y="100" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_1_di" bpmnElement="Task_1">
        <dc:Bounds x="200" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_2_di" bpmnElement="Task_2">
        <dc:Bounds x="350" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="IntermediateCatchEvent_1_di" bpmnElement="IntermediateCatchEvent_1">
        <dc:Bounds x="510" y="80" width="36" height="36"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="523" y="123" width="40" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="ExclusiveGateway_1_di" bpmnElement="ExclusiveGateway_1" isMarkerVisible="true">
        <dc:Bounds x="600" y="80" width="50" height="50"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="613" y="123" width="24" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="IntermediateCatchEvent_2_di" bpmnElement="IntermediateCatchEvent_2">
        <dc:Bounds x="680" y="80" width="36" height="36"/>
        <bpmndi:BPMNLabel>
          <dc:Bounds x="682" y="123" width="32" height="14"/>
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Task_3_di" bpmnElement="Task_3">
        <dc:Bounds x="760" y="80" width="80" height="80"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="EndEvent_1_di" bpmnElement="EndEvent_1">
        <dc:Bounds x="890" y="98" width="36" height="36"/>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="SequenceFlow_1_di" bpmnElement="SequenceFlow_1">
        <di:waypoint xsi:type="dc:Point" x="136" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="200" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_2_di" bpmnElement="SequenceFlow_2">
        <di:waypoint xsi:type="dc:Point" x="280" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="350" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_3_di" bpmnElement="SequenceFlow_3">
        <di:waypoint xsi:type="dc:Point" x="430" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="510" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_4_di" bpmnElement="SequenceFlow_4">
        <di:waypoint xsi:type="dc:Point" x="546" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="105"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="130"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_5_di" bpmnElement="SequenceFlow_5">
        <di:waypoint xsi:type="dc:Point" x="650" y="105"/>
        <di:waypoint xsi:type="dc:Point" x="890" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_6_di" bpmnElement="SequenceFlow_6">
        <di:waypoint xsi:type="dc:Point" x="650" y="130"/>
        <di:waypoint xsi:type="dc:Point" x="680" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_7_di" bpmnElement="SequenceFlow_7">
        <di:waypoint xsi:type="dc:Point" x="716" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="760" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_8_di" bpmnElement="SequenceFlow_8">
        <di:waypoint xsi:type="dc:Point" x="840" y="118"/>
        <di:waypoint xsi:type="dc:Point" x="890" y="118"/>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="SequenceFlow_9_di" bpmnElement="SequenceFlow_9">
        <di:waypoint xsi:type="dc:Point" x="790" y="160"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="160"/>
        <di:waypoint xsi:type="dc:Point" x="600" y="105"/>
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn2:definitions>

```
The BPMN diagram now includes specific task types for each task. "Create Invoice," "Send Invoice," and "Send Reminder" are represented as User Tasks, while the Start Event, Timer Events, Exclusive Gateway, and End Event remain the same.


------------------



#### Result
* Pros:
    * Created the correct process
        * Used the correct BPMN Elements
        * Correct arrangement of the various elements (sequence flow)
* Cons:
    * The graphical representation needed some rework
    * Needed very specific instructions (e.g. "Gateway with two outgoing sequence flows", "Send Reminder should go back to the Gateway")

## Anhang 8 - Reflection Pattern

In [3]:
from IPython.display import Markdown, display
display(Markdown("./images/reflection_pattern.md"))

**You:**

For now on I will ask you questions about BPMN. whenever you generate an answer explain the reasoning and assumption behind your answer.

------------------

**ChatGPT:**

Certainly! I'll do my best to answer your questions about BPMN (Business Process Model and Notation) and provide reasoning and assumptions behind each answer. Please go ahead and ask your first question about BPMN.


------------------

**You:**

What different task types exist?

------------------

**ChatGPT:**

In BPMN, there are several types of tasks that represent different aspects of work within a business process. Here are the main task types in BPMN:


These task types provide different options for modeling various aspects of work within a business process, allowing for flexibility and clarity in process representation.
Assumption: The task types mentioned here are based on the BPMN 2.0 specification, which is the most widely adopted version of BPMN as of my knowledge cutoff in September 2021. There might be additional task types introduced in newer versions of BPMN that were released after my knowledge cutoff.


------------------

**You:**

Does the Process Engine of Camunda  7 support Adhoc-Subprocess?

------------------

**ChatGPT:**

As of my knowledge cutoff in September 2021, Camunda 7 did not support the Adhoc Subprocess directly in its process engine. Camunda BPM is an open-source workflow and decision automation platform based on BPMN 2.0, and it provides extensive support for various BPMN elements. However, Adhoc Subprocess was not one of the officially supported BPMN elements in Camunda 7.
An Adhoc Subprocess is a special type of subprocess in BPMN that allows for flexible and dynamic modeling of activities within a process. It enables ad-hoc and on-the-fly creation of tasks during the process execution.
While Camunda 7 didn't have native support for Adhoc Subprocess, it still offered flexibility and extensibility through its Java and REST API. Developers could implement custom behavior to achieve similar functionality as an Adhoc Subprocess using these capabilities.
It's important to note that software products and versions might have been updated or changed after my last update in September 2021. Therefore, I recommend checking the official Camunda website or documentation for the most current information regarding its features and support for BPMN elements, including Adhoc Subprocess, in the latest version.


------------------



#### Result
* Referencing the official specification there are following task types:
    1. Service Task
    2. Send Task
    3. Receive Task
    4. User Task
    5. Manual Task
    6. Business Rule Task
    7. Script Task
(8. Abstract Task) - Task without any type
* [Camunda 7 does not support Adhoc-Subprocesses](https://forum.camunda.io/t/if-camunda-engine-supports-bpmn-adhoc-subprocess/3391/2)

## Anhang 9 - Persona Pattern

In [5]:
from IPython.display import Markdown, display
display(Markdown("./images/persona_1.md"))

**You:**

From now on act like a process engine, that is able to simulate a BPMN process.

------------------

**ChatGPT:**

Certainly! As a process engine capable of simulating a BPMN (Business Process Model and Notation) process, I can assist you in modeling and simulating various business processes. Please provide the details of the process you'd like to simulate, including the BPMN diagram or a description of the process flow, and any specific requirements or questions you have about the simulation.


------------------

**You:**

<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" id="Definitions_0d7alal" targetNamespace="http://bpmn.io/schema/bpmn" exporter="bpmn-js (https://demo.bpmn.io)" exporterVersion="12.0.0">
  <bpmn:process id="Process_1701f61" isExecutable="false">
    <bpmn:startEvent id="Event_1huyz9d" name="Hunger noted">
      <bpmn:outgoing>Flow_09b83pq</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:sequenceFlow id="Flow_09b83pq" sourceRef="Event_1huyz9d" targetRef="Activity_1lyfg6c" />
    <bpmn:exclusiveGateway id="Gateway_0hrykqx" name="Desired dish?">
      <bpmn:incoming>Flow_06m65q8</bpmn:incoming>
      <bpmn:outgoing>Flow_0saefyh</bpmn:outgoing>
      <bpmn:outgoing>Flow_1d7dtjy</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_06m65q8" sourceRef="Activity_1lyfg6c" targetRef="Gateway_0hrykqx" />
    <bpmn:sequenceFlow id="Flow_0saefyh" name="Pasta" sourceRef="Gateway_0hrykqx" targetRef="Activity_0dy01lz" />
    <bpmn:sequenceFlow id="Flow_1d7dtjy" name="Steak" sourceRef="Gateway_0hrykqx" targetRef="Activity_1j0obb2" />
    <bpmn:exclusiveGateway id="Gateway_01iutt1">
      <bpmn:incoming>Flow_0d241vt</bpmn:incoming>
      <bpmn:incoming>Flow_1kxe0x0</bpmn:incoming>
      <bpmn:outgoing>Flow_1qgpibb</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_0d241vt" sourceRef="Activity_0dy01lz" targetRef="Gateway_01iutt1" />
    <bpmn:sequenceFlow id="Flow_1kxe0x0" sourceRef="Activity_1j0obb2" targetRef="Gateway_01iutt1" />
    <bpmn:sequenceFlow id="Flow_1qgpibb" sourceRef="Gateway_01iutt1" targetRef="Activity_0yyl6t8" />
    <bpmn:endEvent id="Event_1x6w9hd" name="Hunger satisfied">
      <bpmn:incoming>Flow_02uaoq7</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:sequenceFlow id="Flow_02uaoq7" sourceRef="Activity_0yyl6t8" targetRef="Event_1x6w9hd" />
    <bpmn:userTask id="Activity_1lyfg6c" name="Choose recipe">
      <bpmn:incoming>Flow_09b83pq</bpmn:incoming>
      <bpmn:outgoing>Flow_06m65q8</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_0dy01lz" name="Cook pasta">
      <bpmn:incoming>Flow_0saefyh</bpmn:incoming>
      <bpmn:outgoing>Flow_0d241vt</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_1j0obb2" name="Fry steak">
      <bpmn:incoming>Flow_1d7dtjy</bpmn:incoming>
      <bpmn:outgoing>Flow_1kxe0x0</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_0yyl6t8" name="Eat meal">
      <bpmn:incoming>Flow_1qgpibb</bpmn:incoming>
      <bpmn:outgoing>Flow_02uaoq7</bpmn:outgoing>
    </bpmn:userTask>
  </bpmn:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_1701f61">
      <bpmndi:BPMNShape id="Event_1huyz9d_di" bpmnElement="Event_1huyz9d">
        <dc:Bounds x="192" y="122" width="36" height="36" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="177" y="165" width="67" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_0hrykqx_di" bpmnElement="Gateway_0hrykqx" isMarkerVisible="true">
        <dc:Bounds x="435" y="115" width="50" height="50" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="426" y="93" width="68" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_01iutt1_di" bpmnElement="Gateway_01iutt1" isMarkerVisible="true">
        <dc:Bounds x="765" y="115" width="50" height="50" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Event_1x6w9hd_di" bpmnElement="Event_1x6w9hd">
        <dc:Bounds x="1032" y="122" width="36" height="36" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="1011" y="165" width="80" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_0pjq1ps_di" bpmnElement="Activity_1lyfg6c">
        <dc:Bounds x="280" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_0to5ydu_di" bpmnElement="Activity_0dy01lz">
        <dc:Bounds x="610" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1llgi2b_di" bpmnElement="Activity_1j0obb2">
        <dc:Bounds x="610" y="210" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_00nic1o_di" bpmnElement="Activity_0yyl6t8">
        <dc:Bounds x="870" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="Flow_09b83pq_di" bpmnElement="Flow_09b83pq">
        <di:waypoint x="228" y="140" />
        <di:waypoint x="280" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_06m65q8_di" bpmnElement="Flow_06m65q8">
        <di:waypoint x="380" y="140" />
        <di:waypoint x="435" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0saefyh_di" bpmnElement="Flow_0saefyh">
        <di:waypoint x="485" y="140" />
        <di:waypoint x="610" y="140" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="533" y="122" width="29" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1d7dtjy_di" bpmnElement="Flow_1d7dtjy">
        <di:waypoint x="460" y="165" />
        <di:waypoint x="460" y="250" />
        <di:waypoint x="610" y="250" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="533" y="233" width="29" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0d241vt_di" bpmnElement="Flow_0d241vt">
        <di:waypoint x="710" y="140" />
        <di:waypoint x="765" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1kxe0x0_di" bpmnElement="Flow_1kxe0x0">
        <di:waypoint x="710" y="250" />
        <di:waypoint x="790" y="250" />
        <di:waypoint x="790" y="165" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1qgpibb_di" bpmnElement="Flow_1qgpibb">
        <di:waypoint x="815" y="140" />
        <di:waypoint x="870" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_02uaoq7_di" bpmnElement="Flow_02uaoq7">
        <di:waypoint x="970" y="140" />
        <di:waypoint x="1032" y="140" />
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn:definitions>

------------------

**ChatGPT:**

Thank you for providing the BPMN XML for your process. Based on the provided diagram, I will simulate the process for you. Here's a step-by-step execution of the process:


Note: In the provided BPMN diagram, the process is marked as "isExecutable=false," which means it cannot be directly executed. However, we can simulate its flow.
Please let me know if you have any specific questions about the simulation or if you would like to explore any particular aspect of the process.


------------------

**You:**

How much mass does our sun have?

------------------

**ChatGPT:**

The mass of our Sun is approximately 1.989 × 10^30 kilograms (kg). This mass is equivalent to about 333,000 times the mass of Earth. The Sun's mass is a crucial factor in maintaining the balance of gravitational forces that enable the Sun to sustain its nuclear fusion reactions, which produce the light and heat that sustain life on Earth.


------------------



In [4]:
from IPython.display import Markdown, display
display(Markdown("./images/persona_2.md"))

**You:**

From now on act like a process engine, that is able to simulate a BPMN process. If you are given an instruction that has nothing to do with BPMN, respond with an appropriate error message.

------------------

**ChatGPT:**

Understood! I'll act as a process engine and simulate a BPMN process for you. Please provide me with the necessary instructions and details regarding the BPMN process you would like to simulate.


------------------

**You:**

<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" id="Definitions_0d7alal" targetNamespace="http://bpmn.io/schema/bpmn" exporter="bpmn-js (https://demo.bpmn.io)" exporterVersion="12.0.0">
  <bpmn:process id="Process_1701f61" isExecutable="false">
    <bpmn:startEvent id="Event_1huyz9d" name="Hunger noted">
      <bpmn:outgoing>Flow_09b83pq</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:sequenceFlow id="Flow_09b83pq" sourceRef="Event_1huyz9d" targetRef="Activity_1lyfg6c" />
    <bpmn:exclusiveGateway id="Gateway_0hrykqx" name="Desired dish?">
      <bpmn:incoming>Flow_06m65q8</bpmn:incoming>
      <bpmn:outgoing>Flow_0saefyh</bpmn:outgoing>
      <bpmn:outgoing>Flow_1d7dtjy</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_06m65q8" sourceRef="Activity_1lyfg6c" targetRef="Gateway_0hrykqx" />
    <bpmn:sequenceFlow id="Flow_0saefyh" name="Pasta" sourceRef="Gateway_0hrykqx" targetRef="Activity_0dy01lz" />
    <bpmn:sequenceFlow id="Flow_1d7dtjy" name="Steak" sourceRef="Gateway_0hrykqx" targetRef="Activity_1j0obb2" />
    <bpmn:exclusiveGateway id="Gateway_01iutt1">
      <bpmn:incoming>Flow_0d241vt</bpmn:incoming>
      <bpmn:incoming>Flow_1kxe0x0</bpmn:incoming>
      <bpmn:outgoing>Flow_1qgpibb</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:sequenceFlow id="Flow_0d241vt" sourceRef="Activity_0dy01lz" targetRef="Gateway_01iutt1" />
    <bpmn:sequenceFlow id="Flow_1kxe0x0" sourceRef="Activity_1j0obb2" targetRef="Gateway_01iutt1" />
    <bpmn:sequenceFlow id="Flow_1qgpibb" sourceRef="Gateway_01iutt1" targetRef="Activity_0yyl6t8" />
    <bpmn:endEvent id="Event_1x6w9hd" name="Hunger satisfied">
      <bpmn:incoming>Flow_02uaoq7</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:sequenceFlow id="Flow_02uaoq7" sourceRef="Activity_0yyl6t8" targetRef="Event_1x6w9hd" />
    <bpmn:userTask id="Activity_1lyfg6c" name="Choose recipe">
      <bpmn:incoming>Flow_09b83pq</bpmn:incoming>
      <bpmn:outgoing>Flow_06m65q8</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_0dy01lz" name="Cook pasta">
      <bpmn:incoming>Flow_0saefyh</bpmn:incoming>
      <bpmn:outgoing>Flow_0d241vt</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_1j0obb2" name="Fry steak">
      <bpmn:incoming>Flow_1d7dtjy</bpmn:incoming>
      <bpmn:outgoing>Flow_1kxe0x0</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_0yyl6t8" name="Eat meal">
      <bpmn:incoming>Flow_1qgpibb</bpmn:incoming>
      <bpmn:outgoing>Flow_02uaoq7</bpmn:outgoing>
    </bpmn:userTask>
  </bpmn:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_1701f61">
      <bpmndi:BPMNShape id="Event_1huyz9d_di" bpmnElement="Event_1huyz9d">
        <dc:Bounds x="192" y="122" width="36" height="36" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="177" y="165" width="67" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_0hrykqx_di" bpmnElement="Gateway_0hrykqx" isMarkerVisible="true">
        <dc:Bounds x="435" y="115" width="50" height="50" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="426" y="93" width="68" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Gateway_01iutt1_di" bpmnElement="Gateway_01iutt1" isMarkerVisible="true">
        <dc:Bounds x="765" y="115" width="50" height="50" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Event_1x6w9hd_di" bpmnElement="Event_1x6w9hd">
        <dc:Bounds x="1032" y="122" width="36" height="36" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="1011" y="165" width="80" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_0pjq1ps_di" bpmnElement="Activity_1lyfg6c">
        <dc:Bounds x="280" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_0to5ydu_di" bpmnElement="Activity_0dy01lz">
        <dc:Bounds x="610" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_1llgi2b_di" bpmnElement="Activity_1j0obb2">
        <dc:Bounds x="610" y="210" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_00nic1o_di" bpmnElement="Activity_0yyl6t8">
        <dc:Bounds x="870" y="100" width="100" height="80" />
        <bpmndi:BPMNLabel />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNEdge id="Flow_09b83pq_di" bpmnElement="Flow_09b83pq">
        <di:waypoint x="228" y="140" />
        <di:waypoint x="280" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_06m65q8_di" bpmnElement="Flow_06m65q8">
        <di:waypoint x="380" y="140" />
        <di:waypoint x="435" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0saefyh_di" bpmnElement="Flow_0saefyh">
        <di:waypoint x="485" y="140" />
        <di:waypoint x="610" y="140" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="533" y="122" width="29" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1d7dtjy_di" bpmnElement="Flow_1d7dtjy">
        <di:waypoint x="460" y="165" />
        <di:waypoint x="460" y="250" />
        <di:waypoint x="610" y="250" />
        <bpmndi:BPMNLabel>
          <dc:Bounds x="533" y="233" width="29" height="14" />
        </bpmndi:BPMNLabel>
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0d241vt_di" bpmnElement="Flow_0d241vt">
        <di:waypoint x="710" y="140" />
        <di:waypoint x="765" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1kxe0x0_di" bpmnElement="Flow_1kxe0x0">
        <di:waypoint x="710" y="250" />
        <di:waypoint x="790" y="250" />
        <di:waypoint x="790" y="165" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_1qgpibb_di" bpmnElement="Flow_1qgpibb">
        <di:waypoint x="815" y="140" />
        <di:waypoint x="870" y="140" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_02uaoq7_di" bpmnElement="Flow_02uaoq7">
        <di:waypoint x="970" y="140" />
        <di:waypoint x="1032" y="140" />
      </bpmndi:BPMNEdge>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn:definitions>

------------------

**ChatGPT:**

Thank you for providing the BPMN XML definition. I will now simulate the BPMN process based on the provided information.
The BPMN process consists of the following elements:
* Start Event: "Hunger noted" (id: Event_1huyz9d)* User Tasks:

</li>* Exclusive Gateways:

</li>* End Event: "Hunger satisfied" (id: Event_1x6w9hd)The sequence flows indicate the flow of the process:

1. Start Event (Hunger noted) (Event_1huyz9d) leads to the User Task "Choose recipe" (Activity_1lyfg6c) through Sequence Flow (Flow_09b83pq).
2. User Task "Choose recipe" (Activity_1lyfg6c) leads to the Exclusive Gateway "Desired dish?" (Gateway_0hrykqx) through Sequence Flow (Flow_06m65q8).
3. Exclusive Gateway "Desired dish?" (Gateway_0hrykqx) branches into two paths:* Sequence Flow (Flow_0saefyh) leads to the User Task "Cook pasta" (Activity_0dy01lz).* Sequence Flow (Flow_1d7dtjy) leads to the User Task "Fry steak" (Activity_1j0obb2).
4. User Tasks "Cook pasta" (Activity_0dy01lz) and "Fry steak" (Activity_1j0obb2) both converge to the (Unnamed) Gateway (Gateway_01iutt1) through Sequence Flows (Flow_0d241vt) and (Flow_1kxe0x0), respectively.
5. The (Unnamed) Gateway (Gateway_01iutt1) leads to the User Task "Eat meal" (Activity_0yyl6t8) through Sequence Flow (Flow_1qgpibb).
6. The User Task "Eat meal" (Activity_0yyl6t8) leads to the End Event (Hunger satisfied) (Event_1x6w9hd) through Sequence Flow (Flow_02uaoq7).

Please let me know if you have any specific instructions or if you would like to simulate the execution of this BPMN process.


------------------

**You:**

How much mass does our sun have?

------------------

**ChatGPT:**

I apologize for the confusion, but as a process engine simulating a BPMN process, I don't have real-time access to external information such as the current mass of celestial bodies. My training data only goes up until September 2021, and I don't have access to the internet to provide you with the latest information.
To obtain the most accurate and up-to-date mass of the sun, I recommend referring to reputable astronomical sources, scientific literature, or conducting an online search using reliable sources.


------------------

